# Analyzing Ringers Artblocks

This notebook analyzes the Ringers Art Blocks collection with a focus on their traits and the prices they can realize. [Visit the repository](https://github.com/ymyke) for more notebooks on other projects, to use the code, or to contribute.

Important: This notebook uses [Plotly](https://plotly.com/python/) mostly to visualize data. Plotly charts won't be rendered in Githubs notebook viewer. So please [view this notebook on nbviewer](https://nbviewer.org/github/ymyke/niftylytics/blob/main/ringers.ipynb) (or on your local installation after cloning the repo). Note that nbviewer caches notebooks and sometimes takes a long time to pick up a new version.

On the upside, all the Plotly charts are interactive. You can zoom, hover, pan, etc. Zooming is especially important due to the outliers in some the data so you can zoom in on the interesting parts.

In [1]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Get the Ringers

Note that this takes a couple of minutes. Also note the timestamp below,
that is when the information was last retrieved.

In [2]:
import datetime
from opensea import ARTBLOCK_CONTRACT, retrieve_assets

START = 13000000
NUMBER = 1000
IDS = list(range(START, START + NUMBER))
assets = retrieve_assets(token_ids=IDS, contract=ARTBLOCK_CONTRACT)
print(datetime.datetime.now())

.................................................. -- All 1000 assets retrieved.
2021-11-07 16:01:31.384710


## Build a dataframe

In [3]:
import numpy as np
import pandas as pd
from helpers import turn_assets_into_df
df, traits = turn_assets_into_df(assets)
hoverdata = [ "Name", *traits, "Probscore"]

In [4]:
assert set(df.LPsymbol.unique()) == set(["ETH", "WETH", np.nan]), """
This sheet does not do currency conversion at the moment and therefore 
assumes all prices are in (W)ETH. But there are more symbols in the input 
data which would lead to apples being compared to ranges below. Aborting.
"""

## How many pieces have a last price on OpenSea?

In [5]:
print(f"{df[~df.Lastprice.isna()].shape[0] / df.shape[0]:.0%}")

46%


## How are prices evolving over time?

Note that this only takes into account the last sale of each piece.

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20, 8)
pd.plotting.register_matplotlib_converters()

fig = px.scatter(df, x="LPdate", y="Lastprice", hover_data=hoverdata)
fig.show()

## Price levels today

In [7]:
fig = px.histogram(df, x="Lastprice", marginal="box", hover_data=hoverdata)
fig.show()

## How frequent are the different traits?

In [8]:
for trait in traits:
    display(
        pd.DataFrame(df[trait].value_counts(normalize=False, sort=True, ascending=True)).transpose()
    )


,Green,Blue,Black,Beige,Red,Yellow,White
Background,2,11,16,47,54,68,802


,Blue,Red,Yellow,Black,White
Body,6,18,54,402,520


,Green,Blue,Red,N/A
Extra color,4,28,123,845


,55,45,30,44,21,37,24,33,34,27,...,7,10,4,14,8,6,5,25,16,9
Peg count,1,1,1,1,1,2,2,2,2,2,...,33,33,36,52,55,56,64,116,137,146


,"Tiled 5,6",6x6 grid,"Tiled 2,3","Tiled 4,3","Tiled 4,5","Tiled 3,4","Tiled 5,4","Tiled 3,2",Recursive grid,4x4 grid,5x5 grid,3x3 grid
Peg layout,2,3,23,23,24,29,30,33,135,217,236,245


,Smaller near center,Bigger near center,Uniform
Peg scaling,104,124,772


,Bullseye,Solid
Peg style,75,925


,smol boi,Normal
Size,36,964


,Off-center,Balanced
Wrap orientation,212,788


,Weave,Loop
Wrap style,495,505


## How much do people pay for the different traits?

In [9]:
for trait in traits:

    fig = go.Figure()
    for traitvariant in (
        df.groupby(trait).median().sort_values("Lastprice", ascending=False).index
    ):
        fig.add_trace(
            go.Box(
                y=df[df[trait] == traitvariant].Lastprice.values,
                name=traitvariant,
                boxpoints="all",
                jitter=0.2,
                whiskerwidth=0.2,
                marker_size=2,
                line_width=1,
            )
        )
    fig.update_layout(title=trait)
    fig.show()
